In [1]:
import requests
from IPython.core.display import HTML
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import datetime
import os
import re

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
filename="/content/gdrive/My Drive/GG_Project_NLP/Data/commit_messages/json_data.csv"
with open("{}".format(filename), 'r') as csvfile:
  df=pd.read_csv(csvfile)

df["project"]=df["project"].astype(str)

project_list=df["project"].unique().astype(str)
project_list

In [ ]:
u = "http://mail-archives.apache.org/mod_mbox/"
r = requests.get(u)
soup = BeautifulSoup(r.text, 'html.parser')
cols = soup.find_all("td")
pat = re.compile("Author:")
pat2 = re.compile(r'\<(.*?)\>')
#author = set()
#email = set()
commit_data = pd.DataFrame(columns = ['proj_name', 'date_time', 'author_name', 'email_id'])

In [ ]:
for i in range(len(cols)):
  #choosing projects
  proj = cols[i].ul.contents    #enter column index/ loop throught the three columns
  for j in range(len(proj)):
    author = ""
    email = ""
    if proj[j] != '\n':
      proj_name = proj[j].a.get("name").split(".")[0]   #enter row index/ loop through the rows
      print(proj_name)
      if proj_name in project_list: #only scrape 272 projects present in json file
        print(proj_name)
        commit_list = [comm for comm in proj[j].contents[1].text.split() if 'commits' in comm]         
        for commit in commit_list:
          #getting commits - try changing this to get dev, users, etc
          
          u2 = "http://mail-archives.apache.org/mod_mbox/{}-{}/".format(proj_name, commit)
          r2 = requests.get(u2)
          soup2 = BeautifulSoup(r2.text, 'html.parser')

          #choosing the year
          proj2 = soup2.find("table", attrs = {"id" : "grid"})
          tabs = proj2.find_all("table", class_ = "year")

          for k in range(len(tabs)):
            link = tabs[k].tbody.find_all("tr")   #change this index to get different years

            for y in range(len(link)):
              
              mon = link[y].find("span", class_ = "links").a.get("href")
              mon2 = mon.split("/")[0]
              mon = mon2+"/thread"
              #print(mon[:4], mon[4:6])
              nurl = "http://mail-archives.apache.org/mod_mbox/{}-{}/{}".format(proj_name, commit, mon)
              date = "_".join(link[y].td.text.split())
              r3 = requests.get(nurl)
              soup3 = BeautifulSoup(r3.text, 'lxml')
              pages= soup3.body.find("table", id = "msglist").thead.find("th", class_ = "pages").find_all("a")

              if (len(pages)>0):
                  page_list=["http://mail-archives.apache.org/mod_mbox/{}-{}/{}/thread?{}".format(proj_name, commit, mon.split("/")[0], thread) for thread in range(len(pages))]
              else:
                  page_list=["http://mail-archives.apache.org/mod_mbox/{}-{}/{}".format(proj_name, commit, mon)]

              for page in page_list:
                nurl = page
                r3 = requests.get(nurl)
                soup3 = BeautifulSoup(r3.text, 'lxml')
                coms = soup3.body.find("table", id = "msglist").tbody.find_all("tr")
                for c in range(len(coms)):
                #getting different commits and saving them
                  if coms[c].find("td", class_ = "subject").a:
                    string = coms[c].find("td", class_ = "subject").a.get("href")
                    string = string.replace("%3c", "<")
                    string = string.replace("%3e", ">")
                    link2 = string
                    #link2 = "<" + coms[c].find("td", class_ = "subject").a.get("href")[3:-3] + ">"
                    nurl2 = u2 + mon2 + "/" + link2
                    r4 = requests.get(nurl2)
                    soup4 = BeautifulSoup(r4.text, 'lxml')
                    #soup4.tbody.find_all("td", class_ = "right")
                    #soup4.tbody.find("tr", class_ = "contents").pre
                    te = soup4.tbody.find("tr", class_ = "contents").pre.text
                    if 'Author:' in te:
                      split_te = te.split('\n')
                      auth = list(filter(pat.search, split_te))[0]
                      ind_1 = pat.search(auth).span()[1] + 1
                      date_0 = coms[c].find("td", class_ = "date").text.split(',')
                      date_0.insert(2, mon[:4])
                      date_0 = (",").join(date_0)
                      date_time_str = str(date_0)
                      date_time_obj = datetime.datetime.strptime(date_time_str, '%a, %d %b,%Y, %H:%M')
                      #date = "_".join(str(date_time_obj).split())
                      if '<' in auth:
                        bind_1 = pat2.search(auth).span()[0] 
                        bind_2 = pat2.search(auth).span()[1]
                        email = auth[bind_1:bind_2]
                        author = auth[ind_1:bind_1]
                      else:
                        author = auth[ind_1:]
                        email = ""
                      commit_dict = {'proj_name' : proj_name, 'date_time' : date_time_obj, 'author_name' : author, 'email_id' : email}
                      commit_data = commit_data.append(commit_dict, ignore_index=True)
          
          commit_data.to_csv("/content/gdrive/My Drive/GG_Project_NLP/Data/commit_messages/all_pages_author_email/commit_data_iv_{}_{}.csv".format(i),j))
        